In [9]:
from flask import Flask, render_template, Response,redirect, url_for, render_template_string, request, jsonify, redirect
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import datetime
import time
import pymysql
import subprocess
import openai


In [10]:

#점수 조정을 위한 함수 (수정가능)

def sigmoid(x):
    return 1 / (1 + np.exp(-x+1))


In [11]:
#서버 초기 변수들 설정 (db연결)
app = Flask(__name__)

app.secret_key = 'mypassword'  # Flask 세션을 위해 필요

# 점수를 저장할 전역 변수 (선택사항)
score_storage = {}

con = pymysql.connect(host='project-db-campus.smhrd.com', user='sc_24K_bigdata25_p3_4', password='smhrd4',
                      port=3312, db='sc_24K_bigdata25_p3_4', charset='utf8', # 한글처리 (charset = 'utf8')
                      autocommit=True # 결과 DB 반영 (Insert or update)
                     )
cur = con.cursor()

In [12]:
# 카메라 연결 초기값 설정

# 랜드마크 선택
selected_landmarks = [0] + list(range(11, 17)) + list(range(23, 29))
    
# 컬럼 이름 정의
columns = ['frame'] + [f'landmark{idx}_{axis}' for idx in selected_landmarks for axis in ['x', 'y']] + \
              ['neck_x', 'neck_y', 'hip_x', 'hip_y']

cap = cv2.VideoCapture(0)  # 카메라 열기

In [13]:
#함수 정의

def generate_frames(id, activity_at,ps_time_f):

    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

    print('generate_frames 실행' ,id, activity_at)

    output_csv_path = f'./BPE/training/{id}_{activity_at}.csv'
    
    user_id = id
    data = []

    if not cap.isOpened():
        print("카메라를 열 수 없습니다.")
        return


    frame_idx = 0
    start_time = time.time()
    duration = ps_time_f - 5 # 카메라 딜레이를 생각해 2초 적게 
    
    while cap.isOpened() and time.time() - start_time < duration:
        ret, frame = cap.read()
        if not ret:
            print("프레임을 읽을 수 없습니다.")
            break
        frame = cv2.flip(frame,1)
    
        frame_idx += 1
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)
        frame_data = [frame_idx]
    
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            for idx in selected_landmarks:
                landmark = landmarks[idx]
                frame_data.extend([landmark.x, landmark.y])
    
            # 목과 엉덩이 좌표 계산
            neck_x = (landmarks[11].x + landmarks[12].x) / 2
            neck_y = (landmarks[11].y + landmarks[12].y) / 2
            hip_x = (landmarks[23].x + landmarks[24].x) / 2
            hip_y = (landmarks[23].y + landmarks[24].y) / 2
            frame_data.extend([neck_x, neck_y, hip_x, hip_y])
    
        else:
            frame_data.extend([None, None] * len(selected_landmarks))
            frame_data.extend([None, None, None, None])
            
        data.append(frame_data)
    
        # 시각화
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            h, w, _ = frame.shape
    
            # 점 그리기
            for idx in selected_landmarks:
                x, y = int(landmarks[idx].x * w), int(landmarks[idx].y * h)
                cv2.circle(frame, (x, y), 5, (255, 0, 0), -1)  # 파란색 점
    
            # 선 그리기
            for connection in mp_pose.POSE_CONNECTIONS:
                start_idx, end_idx = connection
                if start_idx in selected_landmarks and end_idx in selected_landmarks:
                    start_x, start_y = int(landmarks[start_idx].x * w), int(landmarks[start_idx].y * h)
                    end_x, end_y = int(landmarks[end_idx].x * w), int(landmarks[end_idx].y * h)
                    cv2.line(frame, (start_x, start_y), (end_x, end_y), (255, 255, 255), 2)  # 하얀색 선
    
        _, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()
    
        yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

    pose.close()  # MediaPipe 종료
    df = pd.DataFrame(data, columns=columns)
    df.to_csv(output_csv_path, index=False)
    print(f"Joint 데이터를 {output_csv_path}에 저장했습니다.")

    yield (b'--frame\r\n'
            b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

In [14]:
#db 연결 함수 (select, insert)

def recent_order(id, flow_idx) :
    # activity_at -> datetime으로 바꿀수잇는지 물어보기
    sql = 'select flow_ord from tmp where (id = %s) and (flow_idx = %s) limit 1'
    cur.execute(sql, (id,flow_idx))
    flow_ord = cur.fetchone()

    if flow_ord :
        # if flow_ord[0] == last_ord[0] :
        #     print('시퀀스를 마지막까지 실행한 유저입니다.')
        #     return 1
        # else : 
        print('가장 최근의 flow_ord :', flow_ord[0])
        return flow_ord[0] + 1
    else :
        print('id에 대한 최근 flow_ord가 없습니다.')
        return 1

def flow_last_order(flow_idx) :
    sql = 'select max(flow_ord) from flow_order where flow_idx = %s group by flow_idx'
    cur.execute(sql, (flow_idx,))
    print(flow_idx)
    last_ord = cur.fetchone()
    print('해당 플로우의 마지막 순서:',last_ord)
    return last_ord[0]

def select_ps_data(flow_idx, flow_ord) :
    # flow_ord와 flow_idx에 해당하는 ps_idx를 찾는 쿼리
    sql = 'select ps_idx from flow_order where flow_idx = %s and flow_ord = %s'
    cur.execute(sql, (flow_idx, flow_ord))
    ps_idx = cur.fetchone()  # 쿼리 결과에서 첫 번째 값만 가져옵니다.  
    print(ps_idx)
    return ps_idx

def select_order_data(flow_idx, flow_ord) :
    print('select order data')
    sql = 'select flow_order_idx from flow_order where flow_idx = %s and flow_ord = %s'
    cur.execute(sql, (flow_idx, flow_ord))
    flow_order_idx = cur.fetchone()  # 쿼리 결과에서 첫 번째 값만 가져옵니다.
    print(flow_order_idx)
    return flow_order_idx

def select_flow_data(flow_idx) :
    sql = 'select * from flow_info where flow_idx = %s'
    cur.execute(sql, (flow_idx,))
    flow_info = cur.fetchall()
    print('select flow data 실행')
    return flow_info

def select_pose_data(ps_idx) :
    sql = 'select* from pose_info where ps_idx = %s'
    cur.execute(sql, (ps_idx,))
    ps_info = cur.fetchall()
    print('select pose data 실행')
    return ps_info

def insert_user_data(id, flow_order_idx, activity_at, activity_score) :
    print('insert user data')
    sql = 'insert into user_activity (id, flow_order_idx, activity_at, activity_score) values (%s, %s, %s, %s)'
    cur.execute(sql, (id, flow_order_idx, activity_at, activity_score))
    con.commit()

In [15]:
instruction = """

SYSTEM:
You are an factual AI assistant named '요팡이' who answers the question for a person who is curious about yoga.
Only tasks that HELP learning, such as
1. 요가에 대한 질문 답변 - Questions about yoga
2. 요가 초급자는 '엉덩이를 위한 요가 시퀀스',
'뻣뻣함을 완화하는 목 요가 시퀀스',
'체중 감량을 위한 요가 시퀀스',
'좌골신경통을 예방하기 위한 허리, 햄스트링, 엉덩이를 위한 요가 시퀀스',
'차크라 요가 시퀀스'에서만 추천해줘 - 
Limit yoga beginners to recommend yoga for '엉덩이를 위한 요가 시퀀스',
'뻣뻣함을 완화하는 목 요가 시퀀스',
'체중 감량을 위한 요가 시퀀스',
'좌골신경통을 예방하기 위한 허리, 햄스트링, 엉덩이를 위한 요가 시퀀스',
'차크라 요가 시퀀스'
3. 요가 중급자는 '엉덩이를 여는 요가 시퀀스',
'소화를 위한 요가 시퀀스',
'허리 통증을 위한 요가 시퀀스',
'무릎 관절염을 위한 요가 시퀀스',
'복근과 코어를 위한 요가 시퀀스'에서만 추천해줘 -
Limit yoga middle class to recommend yoga for '엉덩이를 여는 요가 시퀀스',
'소화를 위한 요가 시퀀스',
'허리 통증을 위한 요가 시퀀스',
'무릎 관절염을 위한 요가 시퀀스',
'복근과 코어를 위한 요가 시퀀스'
4. 요가 고급자는 '힙 오프너 요가 시퀀스',
'복근과 코어를 위한 요가 시퀀스',
'팔과 어깨를 위한 요가 시퀀스',
'하트 오픈 및 백벤드 요가 시퀀스',
'피크 포즈 및 분할 포즈 요가 시퀀스',
'간단하고 짧은 스트레칭 요가',
'간단하고 짧은 스트레칭 요가' 에서만 추천해줘 -
Limit yoga seniors to recommend yoga for '힙 오프너 요가 시퀀스',
'복근과 코어를 위한 요가 시퀀스',
'팔과 어깨를 위한 요가 시퀀스',
'하트 오픈 및 백벤드 요가 시퀀스',
'피크 포즈 및 분할 포즈 요가 시퀀스',
'간단하고 짧은 스트레칭 요가',
'간단하고 짧은 스트레칭 요가'
5. 환각 현상 금지 - No hallucinations
6. max_tokens = 500 내에만 대답해
요팡이 answers in KOREAN """

In [ ]:



@app.route('/video_feed/<id>/<activity_at>/<ps_time>')
def video_feed(id, activity_at, ps_time):
    ps_time_f = float(ps_time)
    return Response(generate_frames(id, activity_at,ps_time_f), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/boot/programs/<id>/<flow_idx>')
def index(id, flow_idx):
    
    print('flow_idx:',flow_idx)
    last_order = flow_last_order(flow_idx)
    
    flow_ord = recent_order(id,flow_idx) # flow_idx, id로 들어온 플로우의 몇 번까지 실행했는지
    print('flow_ord:',flow_ord)

    if flow_ord > last_order : #그 플로우의 마지막 플로우 순서 번호보다 크다면 1로 수정
        flow_ord = 1
    
    ps_idx = select_ps_data(flow_idx, flow_ord)
    print('ps_idx:',ps_idx)
    flow_order_idx = select_order_data(flow_idx, flow_ord)
    print('flow_order_idx:',flow_order_idx)


    if flow_ord > last_order :
        flow_ord = 1

    flow_info = select_flow_data(flow_idx[0])
    print(flow_info)
    pose_info = select_pose_data(ps_idx[0])
    print(pose_info)
    
    ps_link = f'{pose_info[0][6]}?autoplay=1'

    activity_at = datetime.datetime.now().strftime('%Y%m%d%H%M%S')

    
    return render_template('programs3.html', id=id,
                           flow_idx=flow_idx,
                           flow_title = flow_info[0][2], 
                           ps_idx = pose_info[0][0],
                           ps_name = pose_info[0][1],  # '웃타나사나'
                           ps_time = pose_info[0][2],  # 30
                           ps_link = ps_link,
                           flow_order_idx = flow_order_idx[0],
                           activity_at = activity_at
                            )
    
    
@app.route('/exit_program')
def exit_program():
    # Spring 서버의 URL로 리다이렉트
    return redirect('http://localhost:8090/boot/programs')

@app.route('/model_run/<id>/<flow_order_idx>/<activity_at>/<flow_idx>/<ps_idx>')
def model_run(id,flow_order_idx, activity_at, flow_idx,ps_idx):
    select_yoga = ps_idx
    # subprocess.run으로 인수 전달
    # result = subprocess.run(["python", "BPE/inference_pair.py", "-v1", f"BPE/training/{id}_{activity_at}.csv", "-v2", f"BPE/rightyogajoint/{select_yoga}.csv"],  # 실행할 명령어
    #result = subprocess.run(["python", "BPE/inference_pair.py", "-v1", "BPE/training/123456_2024-12-11_172246.csv", "-v2", f"BPE/rightyogajoint/{select_yoga}.csv"],  # 실행할 명령어
    #capture_output=True,      # 출력을 캡처
    #text=True                 # 바이트가 아닌 문자열로 출력 받음
    #)
    #BPE 안의 모델 파일 실행 / training 안에 방금 저장된 유저의 파일과 , rightyogajoint에 미리 저장해둔 요가 자세의 올바른 자세 joint 값과 비교 

    #output = result.stdout    
    #실행된 결과에서 score만 가져오기
    #list1 = output.split('\n')
    #score_str = list1[-2]
    # score 점수 조정
    #model_score = ((round(sigmoid((float(score_str.split(' ')[-1])))*10)*10))
    model_score = 40
    activity_at = datetime.datetime.now().strftime('%Y-%m-%d')  # 활동 시간
    activity_score = model_score
    insert_user_data(id, flow_order_idx, activity_at, activity_score)
    
    return render_template('score.html', score=model_score, id=id, flow_idx=flow_idx)

@app.route('/api/message', methods=['POST']) 
def message(): 
    # json 데이터 가져오기
    data = request.get_json() 
    prompt = data.get('message','')

    # openAI ALI로부터 응답 방기
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        max_tokens = 500,
        messages=[ 
            {"role": "system", "content": instruction}, 
            {"role": "user", "content": prompt}
        ]
    )
    
    # 응답에서 'choices' 속성으로 데이터 접근
    message_content = response.choices[0].message.content
    # json 형태로 응답 보내기
    return jsonify({'message': message_content})


# 홈 페이지 렌더링
@app.route('/boot/chat')
def home(): 
    return render_template('chat.html')

@app.route('/exit_chat')
def exit_chat():
    # Spring 서버의 URL로 리다이렉트
    return redirect('http://localhost:8090/boot/mypage')




if __name__ == "__main__":
    app.run(host='localhost', port=8097)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:8097
Press CTRL+C to quit
127.0.0.1 - - [16/Dec/2024 12:45:13] "GET /boot/chat HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2024 12:45:27] "POST /api/message HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2024 12:45:36] "GET /exit_program HTTP/1.1" 302 -


flow_idx: 1
1
해당 플로우의 마지막 순서: (8,)
id에 대한 최근 flow_ord가 없습니다.
flow_ord: 1
(51,)
ps_idx: (51,)
select order data


127.0.0.1 - - [16/Dec/2024 12:45:53] "GET /boot/programs/vncj/1 HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2024 12:45:53] "GET /static/css/index10.css HTTP/1.1" 304 -


(25,)
flow_order_idx: (25,)
select flow data 실행
((1, '초급', '엉덩이를 위한 요가 시퀀스', '엉덩이를 위한 요가 시퀀스는 엉덩이 오프너 범주에 속하며, 학생의 연습 수준에 관계없이 모든 요가 연습의 필수적인 부분을 형성합니다. 엉덩이 오프너 요가 시퀀스는 엉덩이와 허벅지 위쪽에 연결된 근육을 열어 허리 아랫부분에 힘을 제공하기 위해 연습해야 할 기본 시퀀스와 같습니다. 엉덩이의 많은 근육은 엉덩이 관절과 대퇴골에 움직임의 자유, 힘과 안정성을 제공합니다. 따라서 엉덩이 주변 근육을 여는 작업은 대부분의 요가 포즈에 필수적입니다.', '1.png'),)
select pose data 실행
((51, '웃타나사나', 30, '웃타나사나 (Uttanasana)는 요가에서 매우 중요한 앞으로 구부리는 자세입니다. 한국어로는 "서서 Forward Fold" 또는 "서서 고양이 자세"라고도 불리며, "웃타나"는 "강하게" 또는 "강렬하게"라는 뜻을 가진 산스크리트어에서 유래한 단어로, "사나"는 "자세"를 의미합니다. 이 자세는 몸의 앞쪽을 깊게 늘려주는 효과가 있으며, 특히 하체와 척추의 유연성 향상에 매우 유익합니다.', '1-1.jpg', 100, 'https://www.youtube.com/embed/PsU5FKyGH2Y'),)


...

127.0.0.1 - - [16/Dec/2024 13:24:24] "GET /static/js/index10.js HTTP/1.1" 404 -
127.0.0.1 - - [16/Dec/2024 13:24:24] "GET /video_feed/vncj/20241216132423/30 HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2024 13:24:25] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [16/Dec/2024 13:24:31] "GET /exit_program HTTP/1.1" 302 -


flow_idx: 1
1
해당 플로우의 마지막 순서: (8,)
가장 최근의 flow_ord : 1
flow_ord: 2
(50,)
ps_idx: (50,)
select order data


127.0.0.1 - - [16/Dec/2024 13:24:37] "GET /boot/programs/vncj/1 HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2024 13:24:37] "GET /static/css/index10.css HTTP/1.1

(26,)
flow_order_idx: (26,)
select flow data 실행
((1, '초급', '엉덩이를 위한 요가 시퀀스', '엉덩이를 위한 요가 시퀀스는 엉덩이 오프너 범주에 속하며, 학생의 연습 수준에 관계없이 모든 요가 연습의 필수적인 부분을 형성합니다. 엉덩이 오프너 요가 시퀀스는 엉덩이와 허벅지 위쪽에 연결된 근육을 열어 허리 아랫부분에 힘을 제공하기 위해 연습해야 할 기본 시퀀스와 같습니다. 엉덩이의 많은 근육은 엉덩이 관절과 대퇴골에 움직임의 자유, 힘과 안정성을 제공합니다. 따라서 엉덩이 주변 근육을 여는 작업은 대부분의 요가 포즈에 필수적입니다.', '1.png'),)
select pose data 실행
((50, '웃카타사나', 30, '웃카타사나(Utkatasana)는 "의자 자세" 또는 "강력한 자세"로 불리는 요가 자세로, 하체 강화와 균형 감각 향상에 매우 유익한 자세입니다. 이 자세는 다리, 엉덩이, 코어를 강화하는 동시에, 상체의 유연성과 정신 집중을 높이는 데 도움을 줍니다. 웃카타사나는 특히 하체의 근육을 자극하여 다리와 엉덩이의 힘을 길러주고, 심신을 단련하는 데 유용한 자세입니다.', '1-2.jpg', 100, 'https://www.youtube.com/embed/4eWcdcXU3hY'),)
generate_frames 실행 vncj 20241216132437


" 304 -
127.0.0.1 - - [16/Dec/2024 13:24:37] "GET /static/css/fire.css HTTP/1.1" 304 -
127.0.0.1 - - [16/Dec/2024 13:24:37] "GET /static/images/요가시퀀스20.jpg HTTP/1.1" 304 -
127.0.0.1 - - [16/Dec/2024 13:24:37] "GET /video_feed/vncj/20241216132437/30 HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2024 13:24:37] "GET /static/js/index10.js HTTP/1.1" 404 -
127.0.0.1 - - [16/Dec/2024 13:24:38] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [16/Dec/2024 13:24:44] "GET /exit_program HTTP/1.1" 302 -
